In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector as selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2025-03-17 17:55:06.018293: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 17:55:06.760904: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 17:55:07.046588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742234107.431936    7975 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742234107.542612    7975 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 17:55:08.028961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
import pandas as pd
df= pd.read_csv("./data/students_grading.csv")

In [5]:
df.head()

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1


In [7]:
df.dtypes

Student_ID                     object
First_Name                     object
Last_Name                      object
Email                          object
Gender                         object
Age                             int64
Department                     object
Attendance (%)                float64
Midterm_Score                 float64
Final_Score                   float64
Assignments_Avg               float64
Quizzes_Avg                   float64
Participation_Score           float64
Projects_Score                float64
Total_Score                   float64
Grade                          object
Study_Hours_per_Week          float64
Extracurricular_Activities     object
Internet_Access_at_Home        object
Parent_Education_Level         object
Family_Income_Level            object
Stress_Level (1-10)             int64
Sleep_Hours_per_Night         float64
dtype: object

In [9]:
df.isna().sum()/df.shape[0]

Student_ID                    0.0000
First_Name                    0.0000
Last_Name                     0.0000
Email                         0.0000
Gender                        0.0000
Age                           0.0000
Department                    0.0000
Attendance (%)                0.1032
Midterm_Score                 0.0000
Final_Score                   0.0000
Assignments_Avg               0.1034
Quizzes_Avg                   0.0000
Participation_Score           0.0000
Projects_Score                0.0000
Total_Score                   0.0000
Grade                         0.0000
Study_Hours_per_Week          0.0000
Extracurricular_Activities    0.0000
Internet_Access_at_Home       0.0000
Parent_Education_Level        0.3588
Family_Income_Level           0.0000
Stress_Level (1-10)           0.0000
Sleep_Hours_per_Night         0.0000
dtype: float64

In [10]:
df.select_dtypes(include=['object']).nunique()

Student_ID                    5000
First_Name                       8
Last_Name                        6
Email                         5000
Gender                           2
Department                       4
Grade                            5
Extracurricular_Activities       2
Internet_Access_at_Home          2
Parent_Education_Level           4
Family_Income_Level              3
dtype: int64

In [11]:
df.select_dtypes(include=['object']).columns

Index(['Student_ID', 'First_Name', 'Last_Name', 'Email', 'Gender',
       'Department', 'Grade', 'Extracurricular_Activities',
       'Internet_Access_at_Home', 'Parent_Education_Level',
       'Family_Income_Level'],
      dtype='object')

In [15]:
for col in df.select_dtypes(include=['object']).columns:
    if df[col].nunique() < 10:
        print(f"{col:-<50}: {df[col].unique()}")

First_Name----------------------------------------: ['Omar' 'Maria' 'Ahmed' 'John' 'Liam' 'Sara' 'Emma' 'Ali']
Last_Name-----------------------------------------: ['Williams' 'Brown' 'Jones' 'Smith' 'Davis' 'Johnson']
Gender--------------------------------------------: ['Female' 'Male']
Department----------------------------------------: ['Engineering' 'Business' 'Mathematics' 'CS']
Grade---------------------------------------------: ['F' 'A' 'D' 'B' 'C']
Extracurricular_Activities------------------------: ['No' 'Yes']
Internet_Access_at_Home---------------------------: ['Yes' 'No']
Parent_Education_Level----------------------------: ['High School' nan "Master's" 'PhD' "Bachelor's"]
Family_Income_Level-------------------------------: ['Medium' 'Low' 'High']


In [59]:
# encode theTarget "Grade" column
df['Grade'] = df['Grade'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'F': 4})


In [60]:
# drop StudentID, Firstname, email , lastname 
X=df.drop(columns=["Student_ID", "First_Name", "Last_Name", "Email", "Grade"], axis=1)
y=df["Grade"]

In [61]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [62]:
# Define pipelines for numerical and categorical features
# import simpleImputer 
from sklearn.impute import SimpleImputer
# import onehotencoder
from sklearn.preprocessing import OneHotEncoder


num_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),  # Impute missing numerical values with the mean
    StandardScaler()  # Scale numerical features
)

In [63]:
cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),  # Impute missing categorical values with the mode
    OneHotEncoder(handle_unknown='ignore')  # One-hot encode categorical features
)

In [64]:
# Create the column transformer
preprocess = make_column_transformer(
    (num_pipeline, selector(dtype_include="number")),  # Apply num_pipeline to numerical columns
    (cat_pipeline, selector(dtype_include="object")),  # Apply cat_pipeline to categorical columns
    remainder="passthrough"  # Pass through any remaining columns
)


In [65]:
# Apply preprocessing to get transformed data
X_train_transformed = preprocess.fit_transform(X_train)
X_test_transformed = preprocess.transform(X_test)

In [66]:
X_train_transformed.shape

(4000, 29)

In [67]:
X_train_transformed

array([[-0.53804318,  0.87784423,  0.27781892, ...,  0.        ,
         0.        ,  1.        ],
       [-0.03625425, -0.1969664 ,  0.77665382, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.46553468, -0.67123393, -1.10647701, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.96732362,  0.25454216,  0.91735085, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.96732362,  1.10176311, -0.96171024, ...,  0.        ,
         0.        ,  1.        ],
       [-1.54162105, -0.61323527, -1.1547326 , ...,  0.        ,
         0.        ,  1.        ]])

In [68]:
y_train.unique()

array([0, 1, 3, 4, 2])

In [69]:
# 1. Build Your Keras Neural Network
# Define the model
# Determine the number of unique grades
num_unique_grades = len(np.unique(y_train))

# Build the Keras model
model = keras.Sequential([
    layers.Input(shape=(X_train_transformed.shape[1],)),  # Use Input layer here
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_unique_grades, activation='softmax')
])


In [70]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Example learning rate
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [71]:
# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,         # Stop if no improvement after 5 epochs
    restore_best_weights=True  # Restore the best model weights
)

In [72]:
# Train the model with Early Stopping
model.fit(X_train_transformed, y_train, epochs=100, batch_size=32, 
          validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2948 - loss: 1.5412 - val_accuracy: 0.4250 - val_loss: 1.3142
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3891 - loss: 1.3204 - val_accuracy: 0.4250 - val_loss: 1.3091
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4054 - loss: 1.3081 - val_accuracy: 0.4325 - val_loss: 1.3017
Epoch 4/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4220 - loss: 1.2699 - val_accuracy: 0.4175 - val_loss: 1.2975
Epoch 5/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4396 - loss: 1.2568 - val_accuracy: 0.4100 - val_loss: 1.3356
Epoch 6/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4555 - loss: 1.2353 - val_accuracy: 0.3950 - val_loss: 1.3035
Epoch 7/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4690 - loss: 1.2131 - val_accuracy: 0.4050 - val_loss: 1.3139
Epoch 8/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4841 - loss: 1.1807 - val_accuracy: 0.4100

In [73]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_transformed, y_test)
print(f"Accuracy: {accuracy}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3763 - loss: 1.3263 
Accuracy: 0.35899999737739563


In [74]:
# Make predictions (get the predicted classes)
y_pred_probs = model.predict(X_test_transformed)  # Get probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class predictions


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [76]:
# Calculate metrics
from sklearn.metrics import classification_report, recall_score, f1_score

report = classification_report(y_test, y_pred)
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')


In [77]:
# Print metrics
print("Classification Report:\n", report)
print("Recall (Macro):", recall_macro)
print("F1-score (Macro):", f1_macro)

Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.68      0.61       299
           1       0.19      0.14      0.16       195
           2       0.25      0.16      0.19       159
           3       0.27      0.44      0.34       178
           4       0.24      0.15      0.19       169

    accuracy                           0.36      1000
   macro avg       0.30      0.31      0.30      1000
weighted avg       0.33      0.36      0.34      1000

Recall (Macro): 0.31378918114204907
F1-score (Macro): 0.2983486223056081
